In [118]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
from selenium import webdriver
import chromedriver_binary
import lxml
from lxml import html

# Scanning Earnings

This Notebook contains a script that scans https://finance.yahoo.com/calendar/earnings/ for company's that are releasing their earnings today. It will return a dataframe containing the tickers that are releasing their earnings. 

This list of tickers will then subsequently be used to check if any of the stocks are currently consolidating using our function from the "Signals" notebook. 

### Upcoming Earnings for all US Companies

In [160]:
# initialising empty dataframe which will hold all tickers
df = pd.DataFrame()

for i in range(1, 11):

    # the URL 
    url = "https://seekingalpha.com/earnings/earnings-calendar/{}".format(i)

    # Make a GET request to fetch the raw HTML content
    html_content = requests.get(url, headers=headers).text

    # get the HTML content
    soup = BeautifulSoup(html_content, "lxml")

    # obtaining tables
    table = soup.find_all('table')

    # indexing the relevant tables to obtain data of interest
    data = pd.read_html(str(table))[0].set_index("Release Date").iloc[:,:-1]
    
    df = df.append(data)
    
df.head(5)

,Symbol,Name,Release Time
Release Date,,,
09/30/2021,PAYX,"Paychex, Inc.",Pre-Market
09/30/2021,KMX,"CarMax, Inc.",Pre-Market
09/30/2021,ANGO,"AngioDynamics, Inc.",Pre-Market
09/30/2021,JEF,Jefferies Financial Group Inc.,Post-Market
09/30/2021,GHGUF,The Go-Ahead Group plc,Estimated


## Obtaining Similar Data but from MarketWatch

In [159]:
url = "https://www.marketwatch.com/tools/earningscalendar"

# Make a GET request to fetch the raw HTML content
html_content = requests.get(url, headers=headers).text

# get the HTML content
soup = BeautifulSoup(html_content, "lxml")

# obtaining tables
table = soup.find_all('table')

# indexing the relevant tables to obtain data of interest
data = pd.read_html(str(table))

# 9 contains today
df = data[9].set_index("Symbol")

# list of tickers releasing earnings today
today_earnings = [df.index.values[i] for i in range(len(df.index.values))]

## Retrieving Fundamental Data
To-Do:

1) Clean the column names

2) Convert number strings to integers 

In [29]:
stock = "tsla"

inc_url = "https://www.marketwatch.com/investing/stock/{}/financials/income/quarter".format(stock)
bs_url = "https://www.marketwatch.com/investing/stock/{}/financials/balance-sheet/quarter".format(stock)
cf_url = "https://www.marketwatch.com/investing/stock/{}/financials/cash-flow/quarter".format(stock)

# Make a GET request to fetch the raw HTML content
html_content = requests.get(inc_url).text

# get the HTML content
soup = BeautifulSoup(html_content, "lxml")

# obtaining tables
table = soup.find_all('table')

# income statement: 3
# balance sheet: 3, 4
# cash flow: 3, 4, 5

# indexing the relevant tables to obtain data of interest
df = pd.read_html(str(table))[3]

df.head()

,Item Item,30-Jun-2020,30-Sep-2020,31-Dec-2020,31-Mar-2021,30-Jun-2021,5- qtr trend
0,Sales/Revenue Sales/Revenue,6.04B,8.77B,10.74B,10.39B,11.96B,NaN
1,Sales Growth Sales Growth,-,45.31%,22.49%,-3.30%,15.10%,NaN
2,Cost of Goods Sold (COGS) incl. D&A Cost of Go...,4.77B,6.71B,8.68B,8.17B,9.07B,NaN
3,COGS Growth COGS Growth,-,40.66%,29.37%,-5.81%,11.01%,NaN
4,COGS excluding D&A COGS excluding D&A,4.2B,6.12B,8.06B,7.55B,8.39B,NaN
